dnn-hy 实验三 by 孟

针对 origin_airport 和 dest_airport 进行降维，降维方法采用自然语言处理中的 embedding 方法，即将地址名称转变成多维度的量值表示，比如 机场 AMC to [0.5,1.22,3.44,2.8] 这样4维的表示。embedding 后 dot，最后 concate 上天气等信息，形成了一个 end-to-end 的模型， 准确率 70.1%, 并没有质的提升。

先前步骤相同

在 one_hot 编码时没有将 origin_airport 和 dest_airport 编码， 而是将机场名称转变成特定数字表示，比如 AMC 1, DEP 2
这样， 剩下的改变都在模型上体现，也就是 embedding + dot 的网络结构。

In [1]:
import keras

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
from __future__ import print_function

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import seaborn as sns

from sklearn import preprocessing
from sklearn.utils import shuffle 
from sklearn.model_selection import KFold, train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

from keras import initializers
from keras.callbacks import Callback
from keras.layers import Input, Dense, Dropout, Dot, Concatenate, BatchNormalization
from keras.layers.core import Flatten
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras.layers.embeddings import Embedding
from keras.layers import concatenate

%matplotlib inline

In [3]:
data = pd.read_feather('/home/ubuntu/data/flight_data_7_18.feather')

In [4]:
data['feat1'] = data['delay_cat'].map(lambda x: 1 if x == '[-500,30)' or x == '[30,60)' or x == '[60,90)' or x == '[90,120)' else 0)
data['feat2'] = data['feat1'].map(lambda x: 1-x)
data = data.drop(['delay_cat'], axis=1)

In [5]:
data = data.dropna()
#data = shuffle(data)
data = data.sample(frac=0.5)
#print(data.info())

In [6]:
from sklearn.utils import resample

data_majority = data[data.feat1==1]
data_minority = data[data.feat1==0]
data_majority_downsampled = resample(data_majority,
                                     replace=False,    # sample without replacement
                                     n_samples=226020,     # to match minority class
                                     random_state=1) # reproducible results
data_downsampled = pd.concat([data_majority_downsampled, data_minority])
data = data_downsampled

In [7]:
data['sch_time_year'] = data['sch_dep_time'].map(lambda x: x.year)
data['sch_time_month'] = data['sch_dep_time'].map(lambda x: x.month)
data['sch_time_day'] = data['sch_dep_time'].map(lambda x: x.day)
data['sch_time_hour'] = data['sch_dep_time'].map(lambda x: x.hour)

In [8]:
data['sch_time_year'] = (data['sch_time_year'].map(lambda x: int(str(x)[2:])))
month_num = {1:6, 2:2, 3:2, 4:5, 5:0, 6:3, 7:5, 8:1, 9:4, 10:6, 11:2, 12:4}
data['year_num'] = data['sch_time_year'].map(lambda x: (int(x / 4) + x) % 7)
data['month_num']=  data['sch_time_month'].map(lambda x: month_num[x])
data['day_num'] = data['sch_time_day']
data['sch_time_day'] = data.apply(lambda x: (x['year_num'] + x['month_num'] + x['day_num'])%7, axis=1)
time_drop = ['year_num', 'month_num', 'day_num']
data = data.drop(time_drop,axis=1)

In [9]:
data['sch_time_hour'] = data['sch_time_hour'].map(lambda x: 24 if x==0 else x)
for i in range(8):  #每3个小时一个区间
    data['sch_time_hour'] = data['sch_time_hour'].map(lambda x: 0+i if x>0+3*i and x<=3+3*i else x)

In [10]:
lb = ['feat1','feat2']
label = data[lb]
data = data.drop(lb, axis=1)
data = data.drop('sch_dep_time', axis=1)

In [11]:
data1 = data.iloc[:,3:24]

data2 = data.iloc[:,0:3]
data3 = data.iloc[:,24:]
data2 = pd.concat([data2, data3], axis=1)
#print(data1.columns, data2.columns)
data1 = (data1 - data1.mean()) / data1.std()

In [12]:
data = pd.concat([data1, data2], axis=1)

In [13]:
one_hot = ['sch_time_year','sch_time_month','sch_time_day','sch_time_hour','airline']#'origin_airport','dest_airport'
data = pd.get_dummies(data, columns=one_hot)

In [14]:
airport = ['origin_airport', 'dest_airport']
data_1 = data[airport]
data_2 = data.drop(airport, axis=1)

In [15]:
ori_air = set(data_1.origin_airport) 
dest_air = set(data_1.dest_airport)  

In [16]:
dic_ori = {}
dic_ori = dict(zip(ori_air, range(len(ori_air))))
dic_dest = {}
dic_dest = dict(zip(dest_air, range(len(dest_air))))

In [17]:
data_1.origin_airport = data_1.origin_airport.apply(lambda x: dic_ori[x])
data_1.dest_airport = data_1.dest_airport.apply(lambda x: dic_dest[x])

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [18]:
data = pd.concat([data_1, data_2], axis=1)

In [19]:
#data.info()
#label.info()

In [20]:
data_train, data_test, label_train, label_test = train_test_split(data, label, test_size=0.2, random_state=2)

In [21]:
data_train_1 = data_train[airport]
data_train_2 = data_train.drop(airport, axis=1)
data_test_1 = data_test[airport]
data_test_2 = data_test.drop(airport, axis=1)

In [22]:
data_train_1.head()

,origin_airport,dest_airport
592246,51,19
429766,157,164
324894,157,39
389446,166,181
3064883,40,50


In [23]:
Embedding_size = 10
InputSize_OriAir = len(ori_air)
InputSize_DestAir = len(dest_air)
InputSize_NoneAir = len(data_train_2.columns)

inputs_ori = Input(shape=(1,))
inputs_dest = Input(shape=(1,))
inputs_noneair = Input(shape=(InputSize_NoneAir,))

ori_embedding = Embedding(input_dim=InputSize_OriAir, output_dim=Embedding_size)(inputs_ori)
dest_embedding = Embedding(input_dim=InputSize_DestAir, output_dim=Embedding_size)(inputs_dest)

In [24]:
embedding = Dot(axes=2, normalize=True)([ori_embedding, dest_embedding])
flatten_embedding = Flatten()(embedding)

inputs = Concatenate(axis=1)([flatten_embedding, inputs_noneair])
x = Dense(units=128, activation='tanh')(inputs)
x = Dropout(0.1)(x)
x = BatchNormalization()(x)
x = Dense(units=64, activation='relu')(x)
x = Dropout(0.1)(x)
outputs = Dense(units=2, activation='sigmoid')(x)

In [25]:
model = Model(inputs=[inputs_ori, inputs_dest, inputs_noneair], outputs=[outputs])
optimizer = Adam(lr=0.0001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
model.fit(x=[data_train_1.origin_airport, data_train_1.dest_airport, data_train_2],
          y=label_train,
          epochs=100,
          batch_size=256,
          #verbose=2,
          validation_split=0.2
          )

Train on 288999 samples, validate on 72250 samples
Epoch 1/100
288999/288999 [==============================] - 10s 35us/step - loss: 0.6519 - acc: 0.6208 - val_loss: 0.6260 - val_acc: 0.6511
Epoch 2/100
288999/288999 [==============================] - 9s 31us/step - loss: 0.6302 - acc: 0.6459 - val_loss: 0.6154 - val_acc: 0.6601
Epoch 3/100
288999/288999 [==============================] - 9s 31us/step - loss: 0.6194 - acc: 0.6565 - val_loss: 0.6079 - val_acc: 0.6647
Epoch 4/100
288999/288999 [==============================] - 9s 31us/step - loss: 0.6115 - acc: 0.6632 - val_loss: 0.6032 - val_acc: 0.6695
Epoch 5/100
288999/288999 [==============================] - 9s 31us/step - loss: 0.6069 - acc: 0.6680 - val_loss: 0.6002 - val_acc: 0.6732
Epoch 6/100
288999/288999 [==============================] - 9s 31us/step - loss: 0.6030 - acc: 0.6716 - val_loss: 0.5973 - val_acc: 0.6745
Epoch 7/100
288999/288999 [==============================] - 9s 31us/step - loss: 0.5999 - acc: 0.6735 - val

Epoch 59/100
288999/288999 [==============================] - 9s 31us/step - loss: 0.5722 - acc: 0.7012 - val_loss: 0.5762 - val_acc: 0.6973
Epoch 60/100
288999/288999 [==============================] - 9s 31us/step - loss: 0.5718 - acc: 0.7017 - val_loss: 0.5762 - val_acc: 0.6969
Epoch 61/100
288999/288999 [==============================] - 9s 31us/step - loss: 0.5713 - acc: 0.7017 - val_loss: 0.5761 - val_acc: 0.6973
Epoch 62/100
288999/288999 [==============================] - 9s 31us/step - loss: 0.5716 - acc: 0.7011 - val_loss: 0.5759 - val_acc: 0.6967
Epoch 63/100
288999/288999 [==============================] - 9s 31us/step - loss: 0.5712 - acc: 0.7014 - val_loss: 0.5755 - val_acc: 0.6979
Epoch 64/100
288999/288999 [==============================] - 9s 31us/step - loss: 0.5706 - acc: 0.7027 - val_loss: 0.5756 - val_acc: 0.6986
Epoch 65/100
288999/288999 [==============================] - 9s 31us/step - loss: 0.5710 - acc: 0.7023 - val_loss: 0.5752 - val_acc: 0.6978
Epoch 66/100


In [27]:
score = model.evaluate(x=[data_test_1.origin_airport, data_test_1.dest_airport, data_test_2],
                       y=label_test,
                       batch_size=len(data_test))

print('Test score:', score[0])
print('Test accuracy:', score[1])

90313/90313 [==============================] - 0s 2us/step
Test score: 0.5728943943977356
Test accuracy: 0.7011061310768127
